In [23]:
goodnight_moon = [
    "In the great green room There was a telephone And a red balloon",
    "And a picture of-",
    "The cow jumping over the moon",
    "And there were three little bears sitting on chairs And two little kittens",
    "And a pair of mittens",
    "And a little toy house",
    "And a young mouse",
    "And a comb and a brush and a bowl full of mush And a quiet old lady who was whispering “hush” Goodnight room",
    "Goodnight moon",
    "Goodnight cow jumping over the moon Goodnight light",
    "And the red balloon",
    "Goodnight bears",
    "Goodnight chairs",
    "Goodnight kittens",
    "And goodnight mittens",
    "Goodnight clocks",
    "And goodnight socks",
    "Goodnight little house",
    "And goodnight mouse",
    "Goodnight comb",
    "And goodnight brush",
    "Goodnight nobody",
    "Goodnight mush",
    "And goodnight to the old lady whispering “hush” Goodnight stars",
    "Goodnight air",
    "Good night noises everywhere"
]

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

In [4]:
lsa_pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd', TruncatedSVD(n_components=5))
])

In [5]:
lsa_pipe.fit(goodnight_moon)

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ... TruncatedSVD(algorithm='randomized', n_components=5, n_iter=5,
       random_state=None, tol=0.0))])

In [6]:
lsa_pipe

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ... TruncatedSVD(algorithm='randomized', n_components=5, n_iter=5,
       random_state=None, tol=0.0))])

In [7]:
from sklearn.externals import joblib

In [9]:
joblib.dump(lsa_pipe, 'lsa_pipe.p')

['lsa_pipe.p']

In [11]:
fit_lsa_pipe = joblib.load('lsa_pipe.p')

### Put Goodnight Moon in Mongo

In [16]:
import pymongo
cli = pymongo.MongoClient(host='54.201.203.180')
cli.database_names()

['admin',
 'local',
 'my_mongo_database',
 'my_mongo_database_2',
 'my_mongo_database_3',
 'test',
 'udacity']

In [17]:
books = cli.books

In [24]:
goodnight_moon_coll = books.goodnight_moon

In [25]:
goodnight_moon[0]

'In the great green room There was a telephone And a red balloon'

In [27]:
goodnight_moon_coll.insert_one({'text': goodnight_moon[0]})

In [28]:
for text in goodnight_moon[1:]:
    goodnight_moon_coll.insert_one({'text': text})

In [29]:
goodnight_moon_coll.count()

26

In [33]:
curs = goodnight_moon_coll.find()

In [40]:
curs = goodnight_moon_coll.find()
for doc in range(curs.count()):
    this_doc = curs.next()
    this_text = this_doc['text']
    this_text_vec = fit_lsa_pipe.transform([this_text])
    this_doc['text_vec'] = this_text_vec
    goodnight_moon_coll.update({'text':this_text}, this_doc)

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  import sys


InvalidDocument: Cannot encode object: array([[ 0.21464815, -0.10420094,  0.48602431,  0.20660372, -0.04212861]])